In [7]:
import pybullet as p
import pybullet_data
import cv2
import random
import cv2
import numpy as np
import os
import pandas as pd

In [8]:
#define joint ranges
min, max= -np.pi, np.pi
joint_ranges = [(min, max), (min, max), (min, max),(min, max),(min, max),(min, max)]

In [9]:
#Top Camera Setup
width, height = 640, 480
top_camera = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0, 0, 0],distance=2.5,yaw=0, pitch=-90, roll=0, upAxisIndex=2)
top_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0)


In [10]:
# Side Camera Setup
side_camera_position = [2, 0, .5]  # Example position
side_camera_target = [0, 0, .5]      # Point the camera is looking at
side_camera_up_vector = [0, 0, 1]   # Which direction is 'up' for the camera
side_camera = p.computeViewMatrix(cameraEyePosition=side_camera_position, cameraTargetPosition=side_camera_target, cameraUpVector=side_camera_up_vector)

# Define the side camera projection matrix
side_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0)


In [21]:
#Front Camera Setup
front_camera_position = [0, 2, .5]  # Example position
front_camera_target = [0, 0, .5]      # Point the camera is looking at
front_camera_up_vector = [0, 0, 1]   # Which direction is 'up' for the camera
front_camera = p.computeViewMatrix(cameraEyePosition=front_camera_position, cameraTargetPosition=front_camera_target, cameraUpVector=front_camera_up_vector)

# Define the side camera projection matrix
front_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0)

In [23]:
#connect to a pybulllet sim
if p.isConnected():
    p.disconnect()
p.connect(p.GUI)
p.setGravity(0, 0, -9.8)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# plane_id= p.loadURDF("plane.urdf")
arm_id = p.loadURDF("assets/ur5.urdf", [0, 0, 0.4],p.getQuaternionFromEuler([0, 0, 0]))

#create a directory for images
images_dir = "UR5_images_2"
os.makedirs(images_dir, exist_ok=True)

# Initialize DataFrame
columns = ['ImageID', 'Joint1', 'Joint2', 'Joint3', 'Joint4', 'Joint5', 'Joint6']
positions_df = pd.DataFrame(columns=columns)

temp_data = []

for i in range(10000):
    joint_positions= []
    # Set random pose
    for joint_index, (min_val, max_val) in enumerate(joint_ranges):
        random_position = random.uniform(min_val, max_val)
        p.setJointMotorControl2(bodyUniqueId=arm_id,
                                jointIndex=joint_index,
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=random_position)
        joint_positions.append(random_position)
    # Allow some time for the arm to move to the pose
    
    for _ in range(100):
        p.stepSimulation()

    # Capture and save images
    _, _, top_image, _, _ = p.getCameraImage(width, height, top_camera, top_projection)
    _, _, side_image, _, _ = p.getCameraImage(width, height, side_camera, side_projection)
    _, _, front_image, _, _ = p.getCameraImage(width, height, front_camera, front_projection)

    top_image_path = os.path.join(images_dir, f"top_view_{i}.png")
    side_image_path = os.path.join(images_dir, f"side_view_{i}.png")
    front_image_path = os.path.join(images_dir, f"front_view_{i}.png")

    cv2.imwrite(top_image_path, top_image)
    cv2.imwrite(side_image_path, side_image)
    cv2.imwrite(front_image_path, front_image)

    temp_data.append({'ImageID': i, 'Joint1': joint_positions[0], 'Joint2': joint_positions[1], 'Joint3': joint_positions[2], 
                      'Joint4': joint_positions[3], 'Joint5': joint_positions[4], 'Joint6': joint_positions[5]})

# Concatenate the temporary data to the DataFrame
positions_df = pd.concat([positions_df, pd.DataFrame(temp_data)], ignore_index=True)
  
# Append data to DataFrame
positions_df.to_csv('UR5_positions_2.csv', index=False)
